<a href="https://colab.research.google.com/github/p3bble123/ewha16-child-abuse-detection/blob/main/input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 tensorflow-hub opencv-python matplotlib

     |████████████████████████████████| 394.3 MB 13 kB/s 
     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 2.9 MB 59.6 MB/s 
     |████████████████████████████████| 3.8 MB 82.4 MB/s 
     |████████████████████████████████| 462 kB 73.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing insta

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [3]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [5]:
# 어른 아이 구분 함수

def separate_adult(frame, current_keypoints, confidence_threshold):

  # keypoints를 frame 크기에 normalize
  y,x,c = frame.shape
  shaped = np.squeeze(np.multiply(current_keypoints, [y,x,1]))
  
  i = 0
  adult_index = 0
  max_height = 0

  for person in shaped:

    # confidence가 역치 이상일 때 키 추출
    if (person[5][2] > confidence_threshold) and (person[6][2] > confidence_threshold) and (person[11][2] > confidence_threshold) and (person[12][2] > confidence_threshold):
       
       m = (person[5][0]+person[6][0])/2, (person[5][1]+person[6][1])/2      # shoulder 중점
       n = (person[11][0]+person[12][0])/2, (person[11][1]+person[12][1])/2  # hip 중점
       
       # height 구하기 (m과 n 사이의 거리)
       height =  (((n[0]-m[0])**2) + ((n[1]-m[1])**2)) ** 0.5
  
       if height > max_height:
         max_height = height
         adult_index = i

    i += 1

  return adult_index 


In [6]:
def sub(x1,x2):
  return x1[0] - x2[0],x1[1] - x2[1]

In [18]:
# keypoint별 벡터 크기(속력) 구하기
def keypoint_vector(frame, previous_keypoints, current_keypoints, confidence_threshold):
  
  # keypoints를 frame 크기에 normalize
  y,x,c = frame.shape
  c_shaped = np.squeeze(np.multiply(current_keypoints, [y,x,1]))
  p_shaped = np.squeeze(np.multiply(previous_keypoints, [y,x,1]))
  
  # 이전 프레임과 현재 프레임의 Adult keypoints
  p_adult_keypoints = p_shaped[separate_adult(frame, previous_keypoints, 0.3)]
  c_adult_keypoints = c_shaped[separate_adult(frame, current_keypoints, 0.3)]

  # 벡터 차이 구하기
  vector = np.zeros((17,2)) 
  for i in range(17):
    # confidence 역치 이상일 때만
    if (c_adult_keypoints[i][2] >= confidence_threshold) and (p_adult_keypoints[i][2] >= confidence_threshold):
      vector[i][0] = c_adult_keypoints[i][0] - p_adult_keypoints[i][0]
      vector[i][1] = c_adult_keypoints[i][1] - p_adult_keypoints[i][1]
    else:
      vector[i] = [0,0]

  return vector
  

In [32]:
def relationKeypoint(frame,current_keypoints,confidence_threshold,distanceArray):
  # keypoints를 frame 크기에 normalize
  y,x,c = frame.shape
  shaped = np.squeeze(np.multiply(current_keypoints, [y,x,1]))
  
  # 이전 프레임과 현재 프레임의 Adult keypoints
  adult_keypoints = shaped[separate_adult(frame, current_keypoints, 0.3)]

  # 벡터 차이 구하기
  vector = np.zeros((22,1)) 
  #f4
  d4 = distanceArray[0] + distanceArray[2] + distanceArray[3] + distanceArray[5] +distanceArray[6]
  vector[0],vector[1] = sub(adult_keypoints[0],adult_keypoints[1])/d4
  vector[2],vector[3] = sub(adult_keypoints[6],adult_keypoints[1])/d4
  vector[4],vector[5] = sub(adult_keypoints[7],adult_keypoints[1])/d4
  #f5
  d5 = distanceArray[2] + distanceArray[3] + distanceArray[9] + distanceArray[11] 
  vector[6],vector[7] = sub(adult_keypoints[1],adult_keypoints[8])/d5
  vector[8],vector[9] = sub(adult_keypoints[7],adult_keypoints[8])/d5
  vector[10],vector[11] = sub(adult_keypoints[13],adult_keypoints[8])/d5
  #f6
  d6 = distanceArray[5] + distanceArray[6] + distanceArray[10] + distanceArray[12] 
  vector[12],vector[13] = sub(adult_keypoints[0],adult_keypoints[9])/d5
  vector[14],vector[15] = sub(adult_keypoints[6],adult_keypoints[9])/d5
  vector[16],vector[17] = sub(adult_keypoints[12],adult_keypoints[9])/d5
  #f7
  d7 = distanceArray[2] + distanceArray[3] + distanceArray[5] + distanceArray[6] 
  vector[18],vector[19] = sub(adult_keypoints[7],adult_keypoints[0])/d7
  vector[20],vector[21] = sub(adult_keypoints[6],adult_keypoints[0])/d7

  return vector


In [10]:
import math

def dist(x1,x2) -> int:
  x = x1[0]-x2[0]
  y = x1[1]-x2[1]
  return math.sqrt(x**2 + y**2)

In [65]:
def distanceVector(norm_vector):
  distanceArray = np.zeros((13,1))
  
  distanceArray[0] = dist((norm_vector[0],norm_vector[1]),(norm_vector[2],norm_vector[3]))
  distanceArray[1] = dist((norm_vector[2],norm_vector[3]),(norm_vector[6],norm_vector[7]))
  distanceArray[2] = dist((norm_vector[6],norm_vector[7]),(norm_vector[10],norm_vector[11]))
  distanceArray[3] = dist((norm_vector[10],norm_vector[11]),(norm_vector[14],norm_vector[15]))
  distanceArray[4] = dist((norm_vector[2],norm_vector[3]),(norm_vector[4],norm_vector[5]))
  distanceArray[5] = dist((norm_vector[4],norm_vector[5]),(norm_vector[8],norm_vector[9]))
  distanceArray[6] = dist((norm_vector[8],norm_vector[9]),(norm_vector[12],norm_vector[13]))
  distanceArray[7] = dist((norm_vector[2],norm_vector[3]),(norm_vector[18],norm_vector[19]))
  distanceArray[8] = dist((norm_vector[2],norm_vector[3]),(norm_vector[16],norm_vector[17]))
  distanceArray[9] = dist((norm_vector[18],norm_vector[19]),(norm_vector[22],norm_vector[23]))
  distanceArray[10] = dist((norm_vector[16],norm_vector[17]),(norm_vector[20],norm_vector[21]))
  distanceArray[11] = dist((norm_vector[22],norm_vector[23]),(norm_vector[26],norm_vector[27]))
  distanceArray[12] = dist((norm_vector[20],norm_vector[21]),(norm_vector[24],norm_vector[25]))

  return distanceArray

In [62]:
def normalize(frame, previous_keypoints, current_keypoints, confidence_threshold) :
  norm_vector = np.zeros((28,1))
  distanceArray = np.zeros((13,1))
  relationVector = np.zeros((22,1))
  final_vector = np.zeros((50,1))
  vector = keypoint_vector(frame, previous_keypoints, current_keypoints, 0.3)
  #head
  norm_vector[0],norm_vector[1] = (vector[1][0]+vector[2][0])//2,(vector[1][1]+vector[2][1])//2
  #neck
  norm_vector[2],norm_vector[3] = (norm_vector[0]+vector[5][0]+ vector[6][0])//3,(norm_vector[1]+vector[5][1]+ vector[6][1])//3

  #norm_vector로 옮기기 (14포인트)
  for i,points in enumerate(vector):
    x_idx,y_idx = (i-3)*2,((i-3)*2)+1
    norm_vector[x_idx] = points[0]
    norm_vector[y_idx] = points[1]
  
  # 신체 거리 구하기
  
  distanceArray = distanceVector(norm_vector)

  #변위 벡터 계산
  
  #f1
  d1 = distanceArray[0] + distanceArray[3] + distanceArray[6] + distanceArray[11] +distanceArray[12]
  norm_vector[0],norm_vector[1] = norm_vector[0]/d1,norm_vector[1]/d1
  norm_vector[12],norm_vector[13] = norm_vector[12]/d1,norm_vector[13]/d1
  norm_vector[14],norm_vector[15] = norm_vector[14]/d1,norm_vector[15]/d1
  norm_vector[24],norm_vector[25] = norm_vector[24]/d1,norm_vector[25]/d1
  norm_vector[26],norm_vector[27] = norm_vector[26]/d1,norm_vector[27]/d1

  #f2
  d2 = distanceArray[0] + distanceArray[2] + distanceArray[5] + distanceArray[9] +distanceArray[10]
  norm_vector[2],norm_vector[3] = norm_vector[2]/d1,norm_vector[3]/d1
  norm_vector[8],norm_vector[9] = norm_vector[8]/d1,norm_vector[9]/d1
  norm_vector[10],norm_vector[11] = norm_vector[10]/d1,norm_vector[11]/d1
  norm_vector[20],norm_vector[21] = norm_vector[20]/d1,norm_vector[21]/d1
  norm_vector[22],norm_vector[23] = norm_vector[22]/d1,norm_vector[23]/d1
  #f3
  d3 = distanceArray[1] + distanceArray[4] + distanceArray[9] + distanceArray[10] 
  norm_vector[4],norm_vector[5] = norm_vector[4]/d1,norm_vector[5]/d1
  norm_vector[6],norm_vector[7] = norm_vector[6]/d1,norm_vector[7]/d1
  norm_vector[16],norm_vector[17] = norm_vector[16]/d1,norm_vector[17]/d1
  norm_vector[18],norm_vector[19] = norm_vector[18]/d1,norm_vector[19]/d1
  
  relationVector = relationKeypoint(frame,current_keypoints,confidence_threshold,distanceArray)

  final_vector = np.concatenate((norm_vector, relationVector), axis=0)
  return final_vector

In [13]:
from google.colab import drive 
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [69]:
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/gdrive/MyDrive/novak.mp4')
prevImg = None
current_keypoints = None
previous_keypoints = None
input = np.zeros((50,1))

while cap.isOpened():

    ret, frame = cap.read()
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    if prevImg is None:    
      prevImg = input_img

      # Detection section
      results1 = movenet(prevImg)

      # 17개의 키포인트: nose, Leye, Reye, Lear, Rear, Lshoulder, Rshoulder, Lelbow, Relbow, Lwrist, Rwrist, Lhip, Rhip, Lknee, Rknee, Lankle, Rankle
      previous_keypoints = results1['output_0'].numpy()[:,:,:51].reshape((6,17,3))

    else:
      currentImg = input_img

      # Detection section
      results2 = movenet(currentImg)

      # 17개의 키포인트: nose, Leye, Reye, Lear, Rear, Lshoulder, Rshoulder, Lelbow, Relbow, Lwrist, Rwrist, Lhip, Rhip, Lknee, Rknee, Lankle, Rankle
      current_keypoints = results2['output_0'].numpy()[:,:,:51].reshape((6,17,3))

      # 다음 프레임을 위한 프레임 이월
      prevImg = currentImg

      #프레임마다 input계산
      vector = normalize(frame, previous_keypoints, current_keypoints, 0.3)

      #프레임 별로 합치기
      input = np.append(input, vector , axis = 1)
      
      #확인
      print(input.shape)
      

      



    if cv2.waitKey(10) & 0xFF==ord('q'):
        break


cap.release()
cv2.destroyAllWindows()



(50, 2)
(50, 3)
(50, 4)
(50, 5)
(50, 6)
(50, 7)
(50, 8)
(50, 9)
(50, 10)
(50, 11)
(50, 12)
(50, 13)
(50, 14)
(50, 15)
(50, 16)
(50, 17)
(50, 18)
(50, 19)
(50, 20)
(50, 21)
(50, 22)
(50, 23)
(50, 24)
(50, 25)
(50, 26)
(50, 27)
(50, 28)
(50, 29)
(50, 30)
(50, 31)
(50, 32)
(50, 33)
(50, 34)
(50, 35)
(50, 36)
(50, 37)
(50, 38)
(50, 39)
(50, 40)
(50, 41)
(50, 42)
(50, 43)
(50, 44)
(50, 45)
(50, 46)
(50, 47)
(50, 48)
(50, 49)
(50, 50)
(50, 51)
(50, 52)
(50, 53)
(50, 54)
(50, 55)
(50, 56)
(50, 57)
(50, 58)
(50, 59)
(50, 60)
(50, 61)
(50, 62)
(50, 63)
(50, 64)
(50, 65)
(50, 66)
(50, 67)
(50, 68)
(50, 69)
(50, 70)
(50, 71)
(50, 72)
(50, 73)
(50, 74)
(50, 75)
(50, 76)
(50, 77)
(50, 78)
(50, 79)
(50, 80)
(50, 81)
(50, 82)
(50, 83)
(50, 84)
(50, 85)
(50, 86)
(50, 87)
(50, 88)
(50, 89)
(50, 90)
(50, 91)
(50, 92)
(50, 93)
(50, 94)
(50, 95)
(50, 96)
(50, 97)
(50, 98)
(50, 99)
(50, 100)
(50, 101)
(50, 102)
(50, 103)
(50, 104)
(50, 105)
(50, 106)
(50, 107)
(50, 108)
(50, 109)
(50, 110)
(50, 111)
(50, 1

KeyboardInterrupt: ignored

# 새 섹션